In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'kaggle_kkbox_churn_220804.ipynb',
 'kaggle_kkbox_churn_220804.zip',
 'kaggle_kkbox_churn_220805.ipynb',
 'members_v3.csv',
 'train_v2.csv',
 'transactions_v2.csv',
 'user_logs_v2.csv']

In [4]:
df_mem = pd.read_csv("members_v3.csv", low_memory = False)
df_tr = pd.read_csv("train_v2.csv", low_memory = False)

In [5]:
df_mem.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time'],
      dtype='object')

In [6]:
df_tr.columns

Index(['msno', 'is_churn'], dtype='object')

In [7]:
df_tr.head(1)

msno  is_churn
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1

In [10]:
df_join = pd.merge(df_tr, df_mem, on = "msno", how = "inner")
len(df_join)

860967

In [11]:
print(len(df_mem))
print(len(df_tr))

6769473
970960


In [12]:
df_tr["is_churn"].value_counts(normalize = True).round(3)

0    0.91
1    0.09
Name: is_churn, dtype: float64

In [13]:
df_join["is_churn"].value_counts(normalize = True).round(3)

0    0.905
1    0.095
Name: is_churn, dtype: float64

종속변수 불균형(imblanced data)
 - over sampling
 - under sampling

In [14]:
df_join.head(2)

msno  is_churn  city  bd gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28   male   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20   male   

   registered_via  registration_init_time  
0               3                20131223  
1               3                20131223

In [21]:
df_join.columns = list(df_join.columns[:-2]) + ["reg", "date"]
df_join.head(2)

msno  is_churn  city  bd gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28   male   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20   male   

   reg      date  
0    3  20131223  
1    3  20131223

In [22]:
df_join["gender"].isna().sum()

472062

In [23]:
df_join["gender"].unique()

array(['male', nan, 'female'], dtype=object)

In [24]:
df_join["gender"] = df_join["gender"].fillna(-1)
df_join["gender"].unique()

array(['male', -1, 'female'], dtype=object)

In [25]:
df_join["gender"] = df_join["gender"].replace(dict(male = 1,
                                                   female = 2))
df_join["gender"].unique()

array([ 1, -1,  2], dtype=int64)

In [26]:
df_join.head(2)

msno  is_churn  city  bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20       1   

   reg      date  
0    3  20131223  
1    3  20131223

In [27]:
df_join.to_csv("main.csv", index = False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("main.csv")
df.head(2)

msno  is_churn  city  bd  gender  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28       1   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20       1   

   reg      date  
0    3  20131223  
1    3  20131223

In [17]:
df_tr = pd.read_csv("transactions_v2.csv")
df_tr.head(1)

msno  payment_method_id  \
0  ++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=                 32   

   payment_plan_days  plan_list_price  actual_amount_paid  is_auto_renew  \
0                 90              298                 298              0   

   transaction_date  membership_expire_date  is_cancel  
0          20170131                20170504          0

In [11]:
len(df_tr)

In [12]:
df_tr["msno"].nunique()

In [10]:
df_tr_sub = df_tr.loc[df_tr["msno"].isin(df_tr["msno"].unique()[15:17]), ]
# df_tr_sub

### Q1. 사용자별 결제시도 횟수를 구하시오.
※ df_tr_sub 객체를 대상으로 분석하시오.

In [15]:
df_tr_sub["cnt"] = 1
df_tr_sub.groupby("msno")["cnt"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    20
Name: cnt, dtype: int64

In [16]:
df_tr_sub["msno"].value_counts()

+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    20
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     1
Name: msno, dtype: int64

### Q2. 사용자별 주 결제방법을 구하시오.
※ df_tr_sub 객체를 대상으로 분석하시오.

In [21]:
df_tr_sub.groupby("msno")["payment_method_id"].agg(lambda x: x.mode())

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    41
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    41
Name: payment_method_id, dtype: int64

In [40]:
aa = df_tr_sub.groupby("msno")["payment_method_id"].value_counts()
aa = aa.to_frame()
aa.columns = ["cnt"]
aa = aa.reset_index().sort_values("cnt", ascending = False)
aa.groupby("msno").head(1).reset_index(drop = True)

msno  payment_method_id  cnt
0  +3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=                 41   20
1  +2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=                 41    1

### Q3. 결제 상품의 기간을 기준으로 사용자별 결제한 상품의 종류를 계산하시오.
※ 30일 상품만 결제하면 1 / 30일, 7일, 90일 상품을 결제했으면 3

In [43]:
df_tr_sub.groupby("msno")["payment_plan_days"].nunique()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    2
Name: payment_plan_days, dtype: int64

### Q4. 결제 상품의 기간을 모두 합산하시오.
※ 단, is_cancel 을 참고하여 취소분은 제외하시오.

In [47]:
df_tr_sub2 = df_tr_sub.loc[df_tr_sub["is_cancel"] == 0, ]
df_tr_sub2.groupby("msno")["payment_plan_days"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=     30
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    540
Name: payment_plan_days, dtype: int64

### Q5. 7일 이하의 상품 결제 여부를 산출하시오.

In [50]:
df_tr_sub.groupby("msno")["payment_plan_days"].agg(lambda x: sum(x <= 7))

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: payment_plan_days, dtype: int64

### Q6. plan_list_price와 actual_amount_paid 변수를 참고하여 사용자별 실제 지불 총액과 할인된 가격으로 상품을 결제한 횟수를 계산하시오.

In [51]:
df_tr_sub.groupby("msno")["actual_amount_paid"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=      99
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    2931
Name: actual_amount_paid, dtype: int64

In [53]:
df_tr_sub["cnt_dis"] = (df_tr_sub["plan_list_price"] > df_tr_sub["actual_amount_paid"]) + 0
df_tr_sub.groupby("msno")["cnt_dis"].sum()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    0
Name: cnt_dis, dtype: int32

### Q7. 사용자별 자동갱신 결제 여부를 계산하시오.

In [55]:
df_tr_sub.groupby("msno")["is_auto_renew"].max()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    1
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: is_auto_renew, dtype: int64

### Q8. 사용자별 거래 취소 경험 여부를 계산하시오.

In [57]:
df_tr_sub.groupby("msno")["is_cancel"].max()

msno
+2y0N7JdEAgAO2zjWgPHqNHvGZsfXdfiegF7hZW86K8=    0
+3Z/2l0S7ui1s9FlZgsPYPUm3VOBhZGgW5toALP8VOg=    1
Name: is_cancel, dtype: int64

### Q9. Q1 부터 Q8까지 했던 작업을 df_tr 객체를 대상으로 실시하고 그 결과를 전부 취합하여 하나의 데이터프레임으로 정리하시오.
※ 최종 정리 객체명은 df_tr_agg 로 하시오.   
※ df_tr_agg 객체의 row 개수는 1197050 이어야 한다.  
※ Q1: 사용자별 결제시도 횟수  
※ Q2: 사용자별 주 결제방법  
※ Q3: 사용자별 결제한 상품의 종류  
※ Q4: 결제 상품의 기간  
※ Q5: 7일 이하의 상품 결제 여부  
※ Q6: 사용자별 실제 지불 총액과 할인된 가격으로 상품을 결제한 횟수  
※ Q7: 사용자별 자동갱신 결제 여부  
※ Q8: 사용자별 거래 취소 경험 여부

In [58]:
df_tr["msno"].nunique()

1197050

In [59]:
df_tr["cnt"] = 1

In [69]:
df_tr_agg = df_tr.groupby("msno")["cnt"].sum().reset_index()
df_tr_agg.head(2)

msno  cnt
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1

In [70]:
df_tr_q2 = df_tr.groupby("msno")["payment_method_id"].agg(lambda x: x.mode())
df_tr_q2 = df_tr_q2.reset_index()

In [71]:
df_tr_q2.head(2)

msno payment_method_id
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=                22
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=                41

In [72]:
df_tr_agg = df_tr_agg.merge(df_tr_q2, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41

In [73]:
df_tr_q3 = df_tr.groupby("msno")["payment_plan_days"].nunique()
df_tr_q3 = df_tr_q3.reset_index()
df_tr_q3.columns = ["msno", "cnt_p_type"]
df_tr_agg = df_tr_agg.merge(df_tr_q3, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  
0           1  
1           1

In [74]:
df_tr_q4 = df_tr.loc[df_tr["is_cancel"] == 0, ]
df_tr_q4 = df_tr_q4.groupby("msno")["payment_plan_days"].sum()
df_tr_q4 = df_tr_q4.reset_index()
df_tr_q4.columns = ["msno", "p_day_sum"]
df_tr_agg = df_tr_agg.merge(df_tr_q4, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  
0           1      395.0  
1           1       30.0

In [81]:
df_tr_sub = df_tr.loc[df_tr["is_cancel"] == 1, ]
df_tr_sub.head(2)

msno  payment_method_id  \
3    +/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=                 36   
177  /PQk/sPb+NaF4URt2hsJjN99n/RZ05J78CP32jVx8qk=                 41   

     payment_plan_days  plan_list_price  actual_amount_paid  is_auto_renew  \
3                   30              180                 180              1   
177                 30               99                  99              1   

     transaction_date  membership_expire_date  is_cancel  cnt  cnt_dis  
3            20170329                20170331          1    1        0  
177          20150708                20171105          1    1        0

In [83]:
df_tr_sub_c = df_tr.loc[df_tr["msno"].isin(df_tr_sub["msno"]), ]
df_tr_msno_cnt = df_tr_sub_c["msno"].value_counts().reset_index()
df_tr_msno_cnt.loc[df_tr_msno_cnt["msno"] == 1, ]

index  msno
24176  KX+988+/GbcIUwwH5hwi41PROvbEReL34kKEmX1mr9g=     1
24177  XBR6tmgEsMwFez+hU9ERfCPZ4Jxzt8ohgDZmAhvzmGs=     1
24178  MQUkOsIGoKyl2PTLpjLDaG7HVKaQnWvEAHY6W0xbzWo=     1
24179  NCCIxdA8nzCrsVhFO56y3nuLRWqrxtmegZSeFAoeBZk=     1
24180  EQeclepjh08CP+JMN+sseJ8UXIC5B3TAV89A8LtcSe8=     1
...                                             ...   ...
34546  l998p1WAKXMgetk+GCB7MIQIha+atSFKnltYICib2QM=     1
34547  nDRnFV+oMRWbxxXI+b3DpaQn/dCHp0KB5Wm+4b1y17Q=     1
34548  neAvmbQyKhcHLeix6McavenFhNKGVi+2SagYvBdmFyY=     1
34549  oEgl1XF4dBMSqNWS31w8bdmrR68i0Q1ZPkLK0gJUUeA=     1
34550  ymghcYW9EE6VigAFNrdBZNJ+NLNly2FpM35K8mB63n0=     1

[10375 rows x 2 columns]

In [75]:
df_tr_q5 = df_tr.groupby("msno")["payment_plan_days"].agg(lambda x: sum(x <= 7))
df_tr_q5 = df_tr_q5.reset_index()
df_tr_q5.columns = ["msno", "cnt_p_7LE"] # Less than Equal
df_tr_agg = df_tr_agg.merge(df_tr_q5, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  
0           1      395.0          0  
1           1       30.0          0

In [76]:
df_tr_q6 = df_tr.groupby("msno")["actual_amount_paid"].sum()
df_tr_q6 = df_tr_q6.reset_index()
df_tr_q6.columns = ["msno", "sum_p"]
df_tr_agg = df_tr_agg.merge(df_tr_q6, on = "msno", how = "left")

df_tr["cnt_dis"] = (df_tr["plan_list_price"] > df_tr["actual_amount_paid"]) + 0
df_tr_q6 = df_tr.groupby("msno")["cnt_dis"].sum()
df_tr_q6 = df_tr_q6.reset_index()
df_tr_q6.columns = ["msno", "cnt_p_dis"]
df_tr_agg = df_tr_agg.merge(df_tr_q6, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  
0           1      395.0          0   1599          0  
1           1       30.0          0     99          0

In [77]:
df_tr_q7 = df_tr.groupby("msno")["is_auto_renew"].max()
df_tr_q7 = df_tr_q7.reset_index()
df_tr_q7.columns = ["msno", "is_p_ar"]
df_tr_agg = df_tr_agg.merge(df_tr_q7, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  is_p_ar  
0           1      395.0          0   1599          0        0  
1           1       30.0          0     99          0        1

In [78]:
df_tr_q8 = df_tr.groupby("msno")["is_cancel"].max()
df_tr_q8 = df_tr_q8.reset_index()
df_tr_q8.columns = ["msno", "is_p_cancel"]
df_tr_agg = df_tr_agg.merge(df_tr_q8, on = "msno", how = "left")
df_tr_agg.head(2)

msno  cnt payment_method_id  \
0  +++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=    1                22   
1  +++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    1                41   

   cnt_p_type  p_day_sum  cnt_p_7LE  sum_p  cnt_p_dis  is_p_ar  is_p_cancel  
0           1      395.0          0   1599          0        0            0  
1           1       30.0          0     99          0        1            0

In [79]:
len(df_tr_agg)

1197050

In [80]:
df_tr_agg.isna().sum().reset_index()

index      0
0               msno      0
1                cnt      0
2  payment_method_id      0
3         cnt_p_type      0
4          p_day_sum  10376
5          cnt_p_7LE      0
6              sum_p      0
7          cnt_p_dis      0
8            is_p_ar      0
9        is_p_cancel      0

In [84]:
df_tr_agg = df_tr_agg.fillna(0)
df_tr_agg.isna().sum().reset_index()

index  0
0               msno  0
1                cnt  0
2  payment_method_id  0
3         cnt_p_type  0
4          p_day_sum  0
5          cnt_p_7LE  0
6              sum_p  0
7          cnt_p_dis  0
8            is_p_ar  0
9        is_p_cancel  0

In [ ]:
df_tr_agg.to_csv("transactions_agg.csv", index = False)

-----------------------------
# user_log.csv

In [86]:
df_log = pd.read_csv("user_logs_v2.csv")
df_log.head(2)

msno      date  num_25  num_50  \
0  u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=  20170331       8       4   
1  nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=  20170330       2       2   

   num_75  num_985  num_100  num_unq  total_secs  
0       0        1       21       18    6309.273  
1       1        0        9       11    2390.699